# 6조 (17011709 정선아, 17011741 문성용, 17011742 김소영)

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

In [2]:
MovieLens_df = pd.read_table("ratings.dat", sep='::', header=None, names=['UserID','MovieID','Rating','Timestamp'])
MovieLens_df.drop(['Timestamp'], axis=1, inplace=True)
MovieLens_pivot = MovieLens_df.pivot_table(index='UserID', columns='MovieID', values='Rating')

In [3]:
row_index = MovieLens_pivot.isnull().sum(axis = 1).sort_values() # NaN이 적은 순으로 행 뽑아내기
col_index = MovieLens_pivot.isnull().sum(axis = 0).sort_values() # NaN이 적은 순으로 열 뽑아내기

In [4]:
MovieLens_pivot = pd.DataFrame(MovieLens_pivot, index=row_index.index, columns=col_index.index)
MovieLens_pivot.head()

MovieID,2858,260,1196,1210,480,2028,589,2571,1270,593,...,398,1724,642,1386,3237,226,1714,1709,3229,3779
UserID,,,,,,,,,,,,,,,,,,,,,
4169,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
1680,5.0,2.0,5.0,3.0,5.0,5.0,5.0,3.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4277,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1941,1.0,5.0,5.0,5.0,5.0,5.0,3.0,3.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1181,3.0,4.0,4.0,3.0,2.0,4.0,5.0,5.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
MovieLens_pivot.shape

(6040, 3706)

In [6]:
# rating이 많은 사용자 1,000명 / 영화 1,000명 추려내기
MovieLens_pivot = MovieLens_pivot.iloc[:1000,:1000]

In [7]:
MovieLens_pivot.shape

(1000, 1000)

In [8]:
user_index = MovieLens_pivot.index.values # User ID 순서 기억하기
user_index[:5]

array([4169, 1680, 4277, 1941, 1181])

In [9]:
user_index = pd.Series(user_index) # Index rename 인자로 넣기 위해 형태 변경

In [10]:
MovieLens_pivot.fillna(0, inplace=True)
MovieLens_pivot = MovieLens_pivot.astype('int64')

In [11]:
MovieLens_pivot.head()

MovieID,2858,260,1196,1210,480,2028,589,2571,1270,593,...,1438,3528,170,1672,2318,3494,2819,1031,1769,69
UserID,,,,,,,,,,,,,,,,,,,,,
4169,5,5,5,5,5,5,4,4,4,5,...,0,3,4,4,2,3,4,4,3,0
1680,5,2,5,3,5,5,5,3,3,5,...,1,4,4,0,5,3,0,5,0,0
4277,5,5,5,4,5,5,5,5,5,5,...,0,0,4,4,5,5,5,3,4,0
1941,1,5,5,5,5,5,3,3,5,5,...,4,0,3,0,0,3,5,3,3,0
1181,3,4,4,3,2,4,5,5,3,3,...,3,0,2,0,3,3,3,0,4,3


In [12]:
MovieLens_pivot = np.array(MovieLens_pivot) # 함수 인자로 넣기 위해 numpy array 형태로 변경

# 1. Cosine Simiarity

In [13]:
def COS(data):
    size = np.size(data, axis = 0)
    simCOS = np.zeros(shape = (size, size)) # 0으로 초기화 된 행렬 생성
    
    for i in range(0, size): # 각 유저별로 for문 반복
        for j in range(i, size):
            normI = np.linalg.norm(data[i, ]) # i벡터의 크기 계산
            normJ = np.linalg.norm(data[j, ]) # j벡터의 크기 계산
            inputData = np.dot(data[i, ], data[j, ])/(normI * normJ) # Cosine similarity 공식
            simCOS[i, j] = inputData # 행렬에 계산 값 대입하기
            simCOS[j, i] = inputData # 대각선 값 대입하기
    
    return simCOS # 최종 행렬값 반환

In [14]:
COS_result = pd.DataFrame(COS(MovieLens_pivot)) # 결과 행렬을 보기 쉽게 데이터프레임으로 출력
COS_result.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.783954,0.853822,0.787305,0.831981,0.800544,0.810888,0.661759,0.738556,0.826932,...,0.434933,0.473015,0.382136,0.504341,0.462347,0.483454,0.438466,0.441451,0.399943,0.520957
1,0.783954,1.000000,0.779852,0.726713,0.750119,0.706065,0.713478,0.704813,0.658503,0.750136,...,0.460497,0.404052,0.350967,0.467821,0.415196,0.465426,0.409881,0.385530,0.409858,0.429034
2,0.853822,0.779852,1.000000,0.749788,0.828772,0.752793,0.799301,0.703737,0.702768,0.793002,...,0.490387,0.458470,0.312931,0.467175,0.460468,0.510682,0.447116,0.460953,0.327179,0.485313
3,0.787305,0.726713,0.749788,1.000000,0.750639,0.670460,0.717004,0.667341,0.572812,0.700587,...,0.414975,0.489167,0.267407,0.440169,0.330793,0.431054,0.478549,0.358528,0.398493,0.399963
4,0.831981,0.750119,0.828772,0.750639,1.000000,0.770771,0.770291,0.676660,0.713351,0.759169,...,0.465894,0.463363,0.334478,0.475109,0.444315,0.498200,0.443905,0.394250,0.392306,0.509421


In [15]:
COS_result.rename(index=user_index, columns=user_index, inplace=True) # User ID mapping
COS_result

,4169,1680,4277,1941,1181,889,3618,2063,1150,1015,...,3158,5087,4430,1937,131,4560,516,2700,5794,2185
4169,1.000000,0.783954,0.853822,0.787305,0.831981,0.800544,0.810888,0.661759,0.738556,0.826932,...,0.434933,0.473015,0.382136,0.504341,0.462347,0.483454,0.438466,0.441451,0.399943,0.520957
1680,0.783954,1.000000,0.779852,0.726713,0.750119,0.706065,0.713478,0.704813,0.658503,0.750136,...,0.460497,0.404052,0.350967,0.467821,0.415196,0.465426,0.409881,0.385530,0.409858,0.429034
4277,0.853822,0.779852,1.000000,0.749788,0.828772,0.752793,0.799301,0.703737,0.702768,0.793002,...,0.490387,0.458470,0.312931,0.467175,0.460468,0.510682,0.447116,0.460953,0.327179,0.485313
1941,0.787305,0.726713,0.749788,1.000000,0.750639,0.670460,0.717004,0.667341,0.572812,0.700587,...,0.414975,0.489167,0.267407,0.440169,0.330793,0.431054,0.478549,0.358528,0.398493,0.399963
1181,0.831981,0.750119,0.828772,0.750639,1.000000,0.770771,0.770291,0.676660,0.713351,0.759169,...,0.465894,0.463363,0.334478,0.475109,0.444315,0.498200,0.443905,0.394250,0.392306,0.509421
889,0.800544,0.706065,0.752793,0.670460,0.770771,1.000000,0.696477,0.585608,0.725332,0.783714,...,0.452430,0.416717,0.339647,0.518762,0.482262,0.509643,0.443241,0.402925,0.421450,0.562836
3618,0.810888,0.713478,0.799301,0.717004,0.770291,0.696477,1.000000,0.642828,0.634699,0.728053,...,0.423034,0.470908,0.374945,0.471670,0.441478,0.501845,0.407151,0.459760,0.319976,0.423950
2063,0.661759,0.704813,0.703737,0.667341,0.676660,0.585608,0.642828,1.000000,0.497034,0.629814,...,0.501601,0.445116,0.345437,0.425827,0.337016,0.465987,0.387300,0.396997,0.322437,0.296523
1150,0.738556,0.658503,0.702768,0.572812,0.713351,0.725332,0.634699,0.497034,1.000000,0.722812,...,0.383380,0.312233,0.330401,0.478277,0.498289,0.454183,0.312021,0.375015,0.361931,0.596293
1015,0.826932,0.750136,0.793002,0.700587,0.759169,0.783714,0.728053,0.629814,0.722812,1.000000,...,0.466471,0.448151,0.385589,0.493355,0.512415,0.513985,0.416314,0.429085,0.390903,0.523378


# 2. Pearson Correlation Coefficient

In [16]:
def cal_PCC(i, j): # 두 벡터의 PCC값을 계산해 주는 함수
    i = np.array(i) # numpy array로 형변환
    j = np.array(j) # numpy array로 형변환
    
    # 벡터의 0인 원소를 null값으로 바꾸고 null값을 제외한 평균값
    mean_i = np.nanmean(np.where(i != 0, i, np.nan)) 
    mean_j = np.nanmean(np.where(j != 0, j, np.nan))
    
    # 벡터의 원소가 0인 부분의 인덱스를 저장한 리스트 생성
    zero_i = np.where(i == 0)
    zero_j = np.where(j == 0)
    
    # zero_i와 zero_j 벡터를 하나로 합쳐준다.
    zeros = np.concatenate((zero_i, zero_j), axis = None)
    
    # 각 벡터의 원소가 0인 인덱스를 삭제
    del_i = np.delete(i, zeros) 
    del_j = np.delete(j, zeros)
    
    # 각 벡터의 원소에서 평균값을 빼준다.
    del_i = del_i - mean_i
    del_j = del_j - mean_j
    
    return np.dot(del_i, del_j)/(np.linalg.norm(del_i) * np.linalg.norm(del_j))
    # PCC 공식을 통한 similarity 값 반환

def PCC(data):
    size = np.size(data, axis = 0)
    simPCC = np.zeros(shape = (size, size)) # 0으로 초기화 된 행렬 생성
    
    for i in range(0, size):
        for j in range(i, size):
            inputData = cal_PCC(data[i, ], data[j, ]) # cal_PCC함수를 이용
            simPCC[i, j] = inputData
            simPCC[j, i] = inputData
    
    return simPCC

In [17]:
PCC_result = pd.DataFrame(PCC(MovieLens_pivot))
PCC_result.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.224133,0.344098,0.310958,0.280614,0.370986,0.285058,0.144675,0.308620,0.393666,...,0.304496,0.278756,0.366347,0.334140,0.269038,0.121189,0.296278,0.322720,0.359212,0.321835
1,0.224133,1.000000,0.128812,0.067557,0.127791,0.215392,0.025950,0.102322,0.203478,0.284813,...,0.185591,-0.003373,0.141055,0.154531,0.187665,-0.030346,0.171789,0.161977,0.131973,0.210933
2,0.344098,0.128812,1.000000,0.226924,0.268866,0.342965,0.167816,0.192860,0.171031,0.300305,...,0.210410,0.267946,0.370334,0.088242,0.213950,0.292816,0.307191,0.311962,0.352111,0.305310
3,0.310958,0.067557,0.226924,1.000000,0.185667,0.190455,0.238017,0.076582,0.077386,0.125724,...,0.139580,0.480574,0.147858,0.106112,0.083804,0.132437,0.309214,0.094917,0.347435,0.111453
4,0.280614,0.127791,0.268866,0.185667,1.000000,0.325779,0.107800,0.179839,0.219911,0.282240,...,0.226305,0.099787,0.268593,0.228523,0.196040,0.283079,0.233733,0.271397,0.249812,0.284333


In [18]:
PCC_result.rename(index=user_index, columns=user_index, inplace=True)
PCC_result

,4169,1680,4277,1941,1181,889,3618,2063,1150,1015,...,3158,5087,4430,1937,131,4560,516,2700,5794,2185
4169,1.000000,0.224133,0.344098,0.310958,0.280614,0.370986,0.285058,0.144675,0.308620,0.393666,...,0.304496,0.278756,0.366347,0.334140,0.269038,0.121189,0.296278,0.322720,0.359212,0.321835
1680,0.224133,1.000000,0.128812,0.067557,0.127791,0.215392,0.025950,0.102322,0.203478,0.284813,...,0.185591,-0.003373,0.141055,0.154531,0.187665,-0.030346,0.171789,0.161977,0.131973,0.210933
4277,0.344098,0.128812,1.000000,0.226924,0.268866,0.342965,0.167816,0.192860,0.171031,0.300305,...,0.210410,0.267946,0.370334,0.088242,0.213950,0.292816,0.307191,0.311962,0.352111,0.305310
1941,0.310958,0.067557,0.226924,1.000000,0.185667,0.190455,0.238017,0.076582,0.077386,0.125724,...,0.139580,0.480574,0.147858,0.106112,0.083804,0.132437,0.309214,0.094917,0.347435,0.111453
1181,0.280614,0.127791,0.268866,0.185667,1.000000,0.325779,0.107800,0.179839,0.219911,0.282240,...,0.226305,0.099787,0.268593,0.228523,0.196040,0.283079,0.233733,0.271397,0.249812,0.284333
889,0.370986,0.215392,0.342965,0.190455,0.325779,1.000000,0.200924,0.109762,0.339954,0.414480,...,0.196515,0.217359,0.095948,0.271808,0.177613,0.266668,0.378121,0.305127,0.176571,0.347921
3618,0.285058,0.025950,0.167816,0.238017,0.107800,0.200924,1.000000,0.014306,0.135390,0.158523,...,0.205287,0.386274,0.080788,0.234454,0.084594,0.226451,0.308165,0.229263,0.212298,0.009408
2063,0.144675,0.102322,0.192860,0.076582,0.179839,0.109762,0.014306,1.000000,-0.070417,0.162710,...,0.272853,0.083140,0.199343,0.202904,-0.045866,0.152132,0.011032,0.114203,0.098529,0.052915
1150,0.308620,0.203478,0.171031,0.077386,0.219911,0.339954,0.135390,-0.070417,1.000000,0.315365,...,0.378205,0.053843,0.095820,0.280980,0.260734,0.076227,0.018764,0.292244,0.386122,0.306919
1015,0.393666,0.284813,0.300305,0.125724,0.282240,0.414480,0.158523,0.162710,0.315365,1.000000,...,0.242022,0.281961,0.295291,0.279792,0.260986,0.230113,0.289073,0.392262,0.326615,0.274855


# 3. JMSD

In [19]:
def MSD(data):
    size = np.size(data, axis = 0)
    simMSD = np.zeros(shape = (size, size)) # 0으로 초기화 된 행렬 생성
    
    for i in range(0, size):
        for j in range(i, size):
            # 벡터의 원소가 0인 부분을 null값으로 변경
            nozero_i = np.where(data[i, ] == 0, np.nan, data[i, ])
            nozero_j = np.where(data[j, ] == 0, np.nan, data[j, ])
            
            # i벡터에서 j벡터의 각 원소를 빼고 제곱한 결과
            SquaredSum = np.square(nozero_i - nozero_j)
            
            # 제곱한 결과가 null값인 것을 제외
            SquaredSum = SquaredSum[~np.isnan(SquaredSum)]
            
            # MSD 공식에 따른 similarity 값
            AllItems = np.size(SquaredSum, axis=0)
            tmp = np.sum(SquaredSum)/AllItems
            
            # 행렬에 구한 similarity 값 대입
            simMSD[i, j] = tmp
            simMSD[j, i] = simMSD[i, j]
            
    return simMSD

In [20]:
def JAC(data):
    size = np.size(data, axis = 0)
    simJAC = np.zeros(shape = (size, size)) # 0으로 초기화 된 행렬 생성
    
    for i in range(0, size):
        for j in range(i, size):
            # 벡터의 원소가 0이상을 만족하는 인덱스만 1, 0이라면 0
            I = np.array(data[i, ] > 0, dtype = np.int)
            J = np.array(data[j, ] > 0, dtype = np.int)
            
            #I, J벡터의 합
            SumIJ = I + J
            
            # 합한 원소가 1보다 크면 교집합
            Inter = np.sum(np.array(SumIJ > 1, dtype=np.int))
            # 0보다 큰 모든 원소의 합은 합집합
            Union = np.sum(np.array(SumIJ > 0, dtype=np.int))
            
            # JAC 공식을 따른 similarity
            tmp = Inter / Union
            
            # 결과 값 simJAC행렬에 대입
            simJAC[i, j] = tmp
            simJAC[j, i] = simJAC[i, j]
            
    return simJAC

In [21]:
def JMSD(data, max):
    return JAC(data) * (1 - (MSD(data/max))) # JMSD 공식에 따른 similarity, max는 최대 평가 값

In [22]:
JMSD_result = pd.DataFrame(JMSD(MovieLens_pivot, 5))
JMSD_result.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.659609,0.726960,0.660868,0.701589,0.614480,0.658624,0.545407,0.513462,0.656948,...,0.214813,0.241743,0.163288,0.253664,0.195436,0.220292,0.215508,0.199596,0.177643,0.220676
1,0.659609,1.000000,0.626513,0.614995,0.620374,0.530117,0.563279,0.583529,0.457817,0.580714,...,0.247018,0.215609,0.156626,0.241891,0.195825,0.223686,0.211235,0.185973,0.188381,0.199519
2,0.726960,0.626513,1.000000,0.595021,0.653305,0.551529,0.615391,0.529389,0.461179,0.621475,...,0.246293,0.227196,0.119724,0.227887,0.202151,0.238889,0.214299,0.208691,0.145714,0.231020
3,0.660868,0.614995,0.595021,1.000000,0.642845,0.522622,0.576422,0.574730,0.435731,0.536298,...,0.214729,0.251255,0.131429,0.245249,0.156475,0.202461,0.238418,0.174180,0.191244,0.174649
4,0.701589,0.620374,0.653305,0.642845,1.000000,0.612505,0.638793,0.572492,0.543939,0.583044,...,0.217195,0.240277,0.162215,0.249504,0.204223,0.223655,0.225139,0.181689,0.175327,0.220845


In [23]:
JMSD_result.rename(index=user_index, columns=user_index, inplace=True)
JMSD_result

,4169,1680,4277,1941,1181,889,3618,2063,1150,1015,...,3158,5087,4430,1937,131,4560,516,2700,5794,2185
4169,1.000000,0.659609,0.726960,0.660868,0.701589,0.614480,0.658624,0.545407,0.513462,0.656948,...,0.214813,0.241743,0.163288,0.253664,0.195436,0.220292,0.215508,0.199596,0.177643,0.220676
1680,0.659609,1.000000,0.626513,0.614995,0.620374,0.530117,0.563279,0.583529,0.457817,0.580714,...,0.247018,0.215609,0.156626,0.241891,0.195825,0.223686,0.211235,0.185973,0.188381,0.199519
4277,0.726960,0.626513,1.000000,0.595021,0.653305,0.551529,0.615391,0.529389,0.461179,0.621475,...,0.246293,0.227196,0.119724,0.227887,0.202151,0.238889,0.214299,0.208691,0.145714,0.231020
1941,0.660868,0.614995,0.595021,1.000000,0.642845,0.522622,0.576422,0.574730,0.435731,0.536298,...,0.214729,0.251255,0.131429,0.245249,0.156475,0.202461,0.238418,0.174180,0.191244,0.174649
1181,0.701589,0.620374,0.653305,0.642845,1.000000,0.612505,0.638793,0.572492,0.543939,0.583044,...,0.217195,0.240277,0.162215,0.249504,0.204223,0.223655,0.225139,0.181689,0.175327,0.220845
889,0.614480,0.530117,0.551529,0.522622,0.612505,1.000000,0.540690,0.451197,0.517922,0.589593,...,0.237219,0.223833,0.164718,0.290820,0.225504,0.240434,0.246387,0.196640,0.182904,0.260529
3618,0.658624,0.563279,0.615391,0.576422,0.638793,0.540690,1.000000,0.516123,0.481503,0.553573,...,0.218450,0.239066,0.167586,0.251213,0.218187,0.252000,0.212021,0.231069,0.138462,0.192718
2063,0.545407,0.583529,0.529389,0.574730,0.572492,0.451197,0.516123,1.000000,0.388383,0.474946,...,0.258290,0.260950,0.169946,0.227728,0.164167,0.231382,0.209748,0.194040,0.165315,0.139774
1150,0.513462,0.457817,0.461179,0.435731,0.543939,0.517922,0.481503,0.388383,1.000000,0.490601,...,0.191344,0.181274,0.172541,0.275799,0.233684,0.227679,0.171604,0.194126,0.158946,0.268272
1015,0.656948,0.580714,0.621475,0.536298,0.583044,0.589593,0.553573,0.474946,0.490601,1.000000,...,0.246129,0.245459,0.175591,0.252596,0.239781,0.252629,0.213403,0.199895,0.180558,0.253370
